In [ ]:
XTREME_BERT = "microsoft/xtremedistil-l6-h256-uncased"
PYTHIA_14_M = "EleutherAI/pythia-14m"
GPT2 = "openai-community/gpt2"

In [ ]:
model_checkpoint = XTREME_BERT
save_directory = f"models/optimum/{model_checkpoint}"
quant_save_dir = f"{save_directory}/quant"

Hf model to Onnx


In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

# Load a model from transformers and export it to ONNX
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
ort_model = ORTModelForSequenceClassification.from_pretrained(
    model_checkpoint, export=True
)

# Save the ONNX model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Infer Onnx


In [ ]:
from transformers import pipeline
from optimum.onnxruntime import ORTModelForSequenceClassification


def inference(directory, tokenizer, file_name="model.onnx", text="My name is Aman and"):
    model = ORTModelForSequenceClassification.from_pretrained(
        directory, file_name=file_name
    )
    generator = pipeline("text-classification", model=model, tokenizer=tokenizer)
    output = generator(text, max_length=50)

    result = output
    # print(result)
    return result

In [ ]:
inference(save_directory, tokenizer, "model.onnx")

Quantize


In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

quantizer = ORTQuantizer.from_pretrained(save_directory)

dqconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)

model_quantized_path = quantizer.quantize(
    save_dir=quant_save_dir,
    quantization_config=dqconfig,
)

In [ ]:
inference(quant_save_dir, tokenizer, "model_quantized.onnx")